In [1]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv('/content/drive/My Drive/Reviews_slice.csv')

In [3]:
data = data.drop_duplicates()

In [4]:
data = data.dropna()

In [5]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [6]:
from nltk.corpus import stopwords

In [7]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(stop_words)

{'both', "don't", 'hadn', 'should', 'has', 'between', 'above', 'now', 'while', 'doesn', 'if', 'he', 're', "aren't", 'that', 'be', 'd', 'those', 'some', 'my', 'out', 'o', 'yourself', 'just', 'was', 'most', 'haven', "hasn't", "you'd", 'because', 'off', "won't", 'she', 'its', 'll', 'had', 'hers', 'to', 'been', 'did', 'myself', 't', 'shouldn', 'mightn', 'such', 'ourselves', "you've", 'after', 'all', 's', 'will', 'whom', 'only', 'yourselves', 'the', 'below', 'm', 'am', 'what', 'over', "mustn't", 'about', 'it', "weren't", 'mustn', 'yours', 'himself', 'from', 'aren', 'under', 'being', 'hasn', 'and', 'them', 'on', "that'll", 'didn', 'isn', 'but', "mightn't", 'own', 'until', 'who', "isn't", "should've", 'further', "couldn't", 'when', 'up', 'ma', 'other', 'with', "shan't", 'shan', 'then', 'same', 'which', 'through', "haven't", 'y', 'where', "you're", 'wouldn', "doesn't", 'against', "didn't", 'once', 'can', 'or', 'as', "it's", 'needn', 'have', 'an', 'we', 'their', 'are', 'i', 'here', 'of', 'our',

In [8]:
def text_cleaner(text):
    new_string = text.lower()
    new_string = BeautifulSoup(new_string, 'lxml').text
    new_string = re.sub(r'\([^)]*\)', '', new_string)
    new_string = re.sub('"', '', new_string)
    new_string = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_string.split(" ")])
    new_string = re.sub(r"'s\b", "", new_string)
    new_string = re.sub("[^a-zA-Z]", " ", new_string)
    tokens = [w for w in new_string.split() if not w in stop_words]
    long_words = []
    for i in tokens:
        if(len(i) >= 3):
            long_words.append(i)
    return (' '.join(long_words)).strip()

cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t))
    
def summary_cleaner(text):
    new_string = re.sub('"', '', text)
    new_string = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_string.split(' ')])
    new_string = re.sub(r"'s\b", '', new_string)
    new_string = re.sub('[^a-zA-Z]', ' ', new_string)
    new_string = new_string.lower()
    tokens = new_string.split()
    new_string = ''
    for i in tokens:
        if(len(i) > 1):
            new_string = new_string + i + ' '
    return new_string

cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(summary_cleaner(t))

In [9]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary
data['cleaned_summary'].replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [10]:
len(data)

99914

In [11]:
data['cleaned_summary'] = data['cleaned_summary'].apply(lambda x : '_START_' + x + '_END_')

In [12]:
print(data['cleaned_summary'][1])
print(data['cleaned_text'][1])

_START_not as advertised _END_
product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo


In [13]:
max_len = 80
max_len_summary = 10

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data['cleaned_text'], data['cleaned_summary'], test_size = 0.1, random_state = 0, shuffle = True)

In [14]:
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(X_train))
X_train_encoded = x_tokenizer.texts_to_sequences(X_train)
X_train_encoded = pad_sequences(X_train_encoded, maxlen = max_len, padding = 'post')
X_val_encoded = x_tokenizer.texts_to_sequences(X_val)
X_val_encoded = pad_sequences(X_val_encoded, maxlen = max_len, padding = 'post')
x_voc_size = len(x_tokenizer.word_index) + 1

y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_train))
y_train_encoded = y_tokenizer.texts_to_sequences(y_train)
y_val_encoded = y_tokenizer.texts_to_sequences(y_val)
y_train_encoded = pad_sequences(y_train_encoded, maxlen = max_len_summary, padding = 'post')
y_val_encoded = pad_sequences(y_val_encoded, maxlen = max_len_summary, padding = 'post')
y_voc_size = len(y_tokenizer.word_index) + 1

In [15]:
latent_dim = 500

In [26]:
encoder_inputs = layers.Input(shape = (max_len,))
enc_emb = layers.Embedding(x_voc_size, latent_dim, trainable = True)(encoder_inputs)

encoder_lstm1 = layers.LSTM(latent_dim, return_sequences = True, return_state = True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

encoder_lstm2 = layers.LSTM(latent_dim, return_sequences = True, return_state = True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3 = layers.LSTM(latent_dim, return_state = True, return_sequences = True)
encoder_outputs, state_h, state_c = encoder_lstm3(encoder_output2)

decoder_inputs = layers.Input(shape = (None,))
dec_emb_layer = layers.Embedding(y_voc_size, latent_dim, trainable = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = layers.LSTM(latent_dim, return_sequences = True, return_state = True)
decoder_outputs, decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb, initial_state = [state_h, state_c])

attn_layer = layers.Attention()
attn_output = attn_layer([decoder_outputs, encoder_outputs])

decoder_concat_input = layers.Concatenate(axis = -1, name = 'concat_layer')([decoder_outputs, attn_output])

decoder_dense = layers.TimeDistributed(layers.Dense(y_voc_size, activation = 'softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

In [18]:
attn_output.shape

TensorShape([None, None, 500])

In [27]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 80, 500)      25986000    input_9[0][0]                    
__________________________________________________________________________________________________
lstm_16 (LSTM)                  [(None, 80, 500), (N 2002000     embedding_8[0][0]                
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None)]       0                                            
_______________________________________________________________________________________

In [28]:
model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy')
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1)

In [29]:
history = model.fit([X_train_encoded, y_train_encoded[:, :-1]], y_train_encoded.reshape(y_train_encoded.shape[0], y_train_encoded.shape[1], 1)[:, 1:, :], epochs = 50, callbacks = [es], 
                     batch_size = 512, validation_data = ([X_val_encoded, y_val_encoded[:, :-1]], y_val_encoded.reshape(y_val.shape[0], y_val_encoded.shape[1], 1)[:, 1:]))

Epoch 1/50
176/176 [==============================] - 87s 493ms/step - loss: 3.1525 - val_loss: 2.7789
Epoch 2/50
176/176 [==============================] - 86s 487ms/step - loss: 2.6673 - val_loss: 2.5232
Epoch 3/50
176/176 [==============================] - 85s 486ms/step - loss: 2.4661 - val_loss: 2.3881
Epoch 4/50
176/176 [==============================] - 85s 483ms/step - loss: 2.3321 - val_loss: 2.3158
Epoch 5/50
176/176 [==============================] - 85s 483ms/step - loss: 2.2284 - val_loss: 2.2582
Epoch 6/50
176/176 [==============================] - 85s 485ms/step - loss: 2.1366 - val_loss: 2.2202
Epoch 7/50
176/176 [==============================] - 85s 484ms/step - loss: 2.0527 - val_loss: 2.1764
Epoch 8/50
176/176 [==============================] - 85s 485ms/step - loss: 1.9733 - val_loss: 2.1431
Epoch 9/50
176/176 [==============================] - 85s 486ms/step - loss: 1.8970 - val_loss: 2.1249
Epoch 10/50
176/176 [==============================] - 85s 485ms/step - l

In [ ]:
model.get_weights()

In [30]:
reverse_target_word_index = y_tokenizer.word_index
reverse_source_word_index = x_tokenizer.word_index
target_word_index = y_tokenizer.word_index

In [77]:
reverse_target_word_index = y_tokenizer.index_word

In [70]:
decoder_lstm.get_weights()

[array([[ 0.09285009,  0.01536157,  0.0172569 , ..., -0.09856436,
         -0.27567434,  0.11334325],
        [-0.0981838 ,  0.15075378, -0.06689165, ...,  0.13516389,
          0.35894108, -0.1623892 ],
        [ 0.12607414,  0.10584345, -0.13180818, ..., -0.0077575 ,
         -0.06649654, -0.5150198 ],
        ...,
        [-0.1071116 ,  0.01685378,  0.00341256, ...,  0.05633232,
          0.14892916,  0.1066574 ],
        [ 0.02377029,  0.01162305, -0.02431881, ..., -0.05061675,
         -0.10206601, -0.08854173],
        [-0.06321345, -0.03513729,  0.05923058, ...,  0.07705364,
          0.22979861, -0.18189426]], dtype=float32),
 array([[ 0.03627593,  0.42709506,  0.04562321, ...,  0.6294713 ,
          0.36680633, -0.2599982 ],
        [ 0.06720988,  0.05094253, -0.01364066, ..., -0.0529791 ,
          0.01087383, -0.00882576],
        [ 0.04186444, -0.01414681,  0.21144482, ..., -0.04258341,
          0.00069788,  0.12477817],
        ...,
        [ 0.03688895, -0.00161205,  0.0

In [ ]:
encoder_model = Model(inputs = encoder_inputs, outputs = [encoder_outputs, state_h, state_c])

In [71]:
decoder_state_input_h = layers.Input(shape = (latent_dim,))
decoder_state_input_c = layers.Input(shape = (latent_dim,))
decoder_hidden_state_input = layers.Input(shape = (max_len, latent_dim))

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = [decoder_state_input_h, decoder_state_input_c])

attn_out_inf = attn_layer([decoder_outputs2, decoder_hidden_state_input])
decoder_inf_concat = layers.Concatenate(axis = -1, name = 'concat')([decoder_outputs2, attn_out_inf])

decoder_outputs2 = decoder_dense(decoder_inf_concat)

decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
                      [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
decoder_model.get_weights()

In [35]:
test_sequence = data['cleaned_text'][9876]

In [39]:
test_tokens = x_tokenizer.texts_to_sequences([test_sequence])
test_seq = pad_sequences(test_tokens, maxlen = max_len, padding = 'post')

In [43]:
e_out, e_h, e_c = encoder_model.predict(test_seq)

In [44]:
target_seq = np.zeros((1, 1))

In [47]:
target_seq[0, 0] = target_word_index['start']

In [ ]:
reverse_target_word_index

In [50]:
target_seq

array([[1.]])

In [78]:
stop_condition = False
decoded_sentence = ''
while not stop_condition:
  output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
  sampled_token_index = np.argmax(output_tokens[0, -1, :])
  sampled_token = reverse_target_word_index[sampled_token_index]
  if (sampled_token != 'end'):
    decoded_sentence += " "+sampled_token
  if (sampled_token == 'end' or len(decoded_sentence.split()) >= (max_len_summary-1)):
    stop_condition = True
  target_seq = np.zeros((1, 1))
  target_seq[0, 0] = sampled_token_index
  e_h, e_c = h, c

In [79]:
print(decoded_sentence)

 great gift


In [81]:
print(data['Text'][9876])

I got these for my boyfriend for Christmas. They are a bit smaller then what i figured they would be from the box, I havent given them to him yet of course but i am sure we will have fun trying them. The box came two day earlier then what amazon said they would come which is nice :)


In [83]:
print(data['Summary'][9876])

very cool gift


In [84]:
model.save_weights('text_model.h5')